In [ ]:
# ! rm -r /kaggle/working/

In [ ]:
!pip install -q ivy

# !git clone https://github.com/unifyai/models.git
# !cd models && python3 -m pip install -q -e .
!python3 -m pip install torchvision

In [1]:
import ivy
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')

In [4]:
class ConvBlock(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(ConvBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding, with_bias=False, data_format="NCHW")
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001, data_format="NCS")

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = ivy.relu(x)
        return x

In [7]:
def test_conv_block():
    # testing with 1st ConvBlock configs in the googlenet block
    # ie, self.conv1 = ConvBlock(3, 64, [7,7], 2, padding=3)
    # random_test_tensor = ivy.random_normal(shape=(1, 224, 224, 3))
    random_test_tensor = ivy.random_normal(shape=(1, 3, 224, 224))

    display(f"random_test_tensor shape is: {random_test_tensor.shape}")
    
    conv_block = ConvBlock(3, 64, [7,7], 2, padding=3)
    calc_output_shape = conv_block(random_test_tensor).shape
    # true_output_shape = (1, 112, 112, 64)
    true_output_shape = (1, 64, 112, 112)

    display(f"calc_output_shape is: {calc_output_shape} | and true_output_shape is: {true_output_shape}")
    
    assert calc_output_shape == true_output_shape;

test_conv_block()

'random_test_tensor shape is: ivy.Shape(1, 3, 224, 224)'

'calc_output_shape is: ivy.Shape(1, 64, 112, 112) | and true_output_shape is: (1, 64, 112, 112)'

In [13]:
class Inception(ivy.Module):
    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        self.in_channels = in_channels
        self.num1x1 = num1x1
        self.num3x3_reduce = num3x3_reduce
        self.num3x3 = num3x3
        self.num5x5_reduce = num5x5_reduce
        self.num5x5 = num5x5
        self.pool_proj = pool_proj
        super(Inception, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv_1x1 = ConvBlock(self.in_channels, self.num1x1, kernel_size=[1, 1], stride=1, padding=0)
                                    
        self.conv_3x3 = ConvBlock(self.in_channels, self.num3x3_reduce, kernel_size=[1, 1], stride=1, padding=0)
        self.conv_3x3_red = ConvBlock(self.num3x3_reduce, self.num3x3, kernel_size=[3, 3], stride=1, padding=1)
                                   
        self.conv_5x5 = ConvBlock(self.in_channels, self.num5x5_reduce, kernel_size=[1, 1], stride=1, padding=0)
        self.conv_5x5_red = ConvBlock(self.num5x5_reduce, self.num5x5, kernel_size=[3, 3], stride=1, padding=1)
                                        
        self.pool_proj_conv = ConvBlock(self.in_channels, self.pool_proj, kernel_size=[1, 1], stride=1, padding=0)
                                        
    def _forward(self, x):
        # unit testing using ==> """self.inception3a = inception_block(192, 64, 96, 128, 16, 32, 32)"""
        display('#######################################################################################')
        # 1x1
        display(f"the input shape should be (1, 28, 28, 192) | and it is: {ivy.shape(x)}")
        conv_1x1 = self.conv_1x1(x)
        
        # 3x3
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        conv_3x3 = self.conv_3x3(x)
        
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        conv_3x3_red = self.conv_3x3_red(conv_3x3)
        
        # 5x5
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        conv_5x5 = self.conv_5x5(x)
        
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        conv_5x5_red = self.conv_5x5_red(conv_5x5)
        
        # pool_proj
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        pool_proj = ivy.max_pool2d(x, [3,3], 1, 1, ceil_mode=True, data_format="NCHW")
        
        display(f"the input shape should be () | and it is: {ivy.shape(x)}")
        pool_proj = self.pool_proj_conv(pool_proj)
        
        display(f"the input shape should be () | and it is: {ivy.shape(conv_1x1)}")
        display(f"the input shape should be () | and it is: {ivy.shape(conv_3x3_red)}")
        display(f"the input shape should be () | and it is: {ivy.shape(conv_5x5_red)}")
        display(f"the input shape should be (1, 1024) | and it is: {ivy.shape(pool_proj)}")
        ret = ivy.concat([conv_1x1, conv_3x3_red, conv_5x5_red, pool_proj], axis=1)
        display(f"the output shape should be (1, 28, 28, 256) | and it is: {ivy.shape(x)}")
        display('#######################################################################################')
        
        return ret

In [16]:
def test_inception_block():
    # testing with 1st ConvBlock configs in the googlenet block
    # ie, self.inception3A = Inception(192, 64, 96, 128, 16, 32, 32)
    # random_test_tensor = ivy.random_normal(shape=(1, 28, 28, 192))
    random_test_tensor = ivy.random_normal(shape=(1, 192, 28, 28))

    display(f"random_test_tensor shape is: {random_test_tensor.shape}")
    
    inception_block = Inception(192, 64, 96, 128, 16, 32, 32)
    calc_output_shape = inception_block(random_test_tensor).shape
    # true_output_shape = (1, 28, 28, 256)
    true_output_shape = (1, 256, 28, 28)

    display(f"calc_output_shape is: {calc_output_shape} | and true_output_shape is: {true_output_shape}")
    
    assert calc_output_shape == true_output_shape;

test_inception_block()

'random_test_tensor shape is: ivy.Shape(1, 192, 28, 28)'

'#######################################################################################'

'the input shape should be (1, 28, 28, 192) | and it is: ivy.Shape(1, 192, 28, 28)'

'the input shape should be () | and it is: ivy.Shape(1, 192, 28, 28)'

'the input shape should be () | and it is: ivy.Shape(1, 192, 28, 28)'

...

'the input shape should be () | and it is: ivy.Shape(1, 32, 28, 28)'

'the input shape should be (1, 1024) | and it is: ivy.Shape(1, 32, 28, 28)'

'the output shape should be (1, 28, 28, 256) | and it is: ivy.Shape(1, 192, 28, 28)'

'#######################################################################################'

'calc_output_shape is: ivy.Shape(1, 256, 28, 28) | and true_output_shape is: (1, 256, 28, 28)'

In [23]:
class Auxiliary(ivy.Module):
    def __init__(self, in_channels, num_classes):
        self.in_channels = in_channels
        self.num_classes = num_classes
        super(Auxiliary, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ConvBlock(self.in_channels, 128, [1,1], 1, 0)
        
        self.fc1 = ivy.Linear(2048, 1024)

        self.dropout = ivy.Dropout(0.7)
        
        self.fc2 = ivy.Linear(1024, self.num_classes)
        self.softmax = ivy.Softmax()
        
    def _forward(self, x):
        # adap avg pool layer
        # display(f"the input shape for permute_dim layer should be (1, 14, 14, 512) | and it is: {ivy.shape(x)}")
        # out = ivy.permute_dims(x, (0, 3, 1, 2))
        
        display(f"the input shape for adapAvgPool layer should be (1, 512, 14, 14) | and it is: {ivy.shape(x)}")
        out = ivy.adaptive_avg_pool2d(x, [4,4])
        
        # display(f"the input shape for permute_dim layer should be (1, 14, 14, 512) | and it is: {ivy.shape(x)}")
        # out = ivy.permute_dims(out, (0, 2, 3, 1))
        
        # conv
        display(f"the input shape for conv layer should be (1, 4, 4, 512) | and it is: {ivy.shape(out)}")
        out = self.conv(out)
        
        # flatten
        display(f"the input shape for flatten layer should be (1, 4, 4, 128) | and it is: {ivy.shape(out)}")
        out = ivy.flatten(out, start_dim=1)
        
        # fc1
        display(f"the input shape for fc1 layer should be (1, 2048) | and it is: {ivy.shape(out)}")        
        out = self.fc1(out)
        
        display(f"the input shape for relu layer should be (1, 1024) | and it is: {ivy.shape(out)}")
        out= ivy.relu(out)

        # dropout
        display(f"the input shape for dropout layer should be (1, 1024) | and it is: {ivy.shape(out)}")
        out = self.dropout(out)
        
        # fc2
        display(f"the input shape for fc2 layer should be (1, 1024) | and it is: {ivy.shape(out)}")
        out = self.fc2(out)
        display(f"the output shape from fc2 layer should be (1, 1000) | and it is: {ivy.shape(out)}")
        
        return out

In [24]:
def test_auxiliary_block():
    # testing with 1st ConvBlock configs in the googlenet block
    # ie, self.aux4A = Auxiliary(512, self.num_classes)
    # random_test_tensor = ivy.random_normal(shape=(1, 14, 14, 512))
    random_test_tensor = ivy.random_normal(shape=(1, 512, 14, 14))

    display(f"random_test_tensor shape is: {random_test_tensor.shape}")
    
    auxiliary_block = Auxiliary(512, 1000)
    display(f"Auxiliary block configuration is: Auxiliary(512, 1000)")
    calc_output_shape = auxiliary_block(random_test_tensor).shape
    true_output_shape = (1, 1000)
    display(f"calc_output_shape is: {calc_output_shape} | and true_output_shape is: {true_output_shape}")
    
    assert calc_output_shape == true_output_shape;

test_auxiliary_block()

'random_test_tensor shape is: ivy.Shape(1, 512, 14, 14)'

'Auxiliary block configuration is: Auxiliary(512, 1000)'

'the input shape for adapAvgPool layer should be (1, 512, 14, 14) | and it is: ivy.Shape(1, 512, 14, 14)'

'the input shape for conv layer should be (1, 4, 4, 512) | and it is: ivy.Shape(1, 512, 4, 4)'

'the input shape for flatten layer should be (1, 4, 4, 128) | and it is: ivy.Shape(1, 128, 4, 4)'

...

'the input shape for relu layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the input shape for dropout layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the input shape for fc2 layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the output shape from fc2 layer should be (1, 1000) | and it is: ivy.Shape(1, 1000)'

'calc_output_shape is: ivy.Shape(1, 1000) | and true_output_shape is: (1, 1000)'

In [25]:
class GoogLeNet(ivy.Module):
    def __init__(self, num_classes=1000, v: ivy.Container = None,):
        if v is not None:
            self.v = v
        self.num_classes = num_classes
        super(GoogLeNet, self).__init__(v=v)

    def _build(self, *args, **kwargs):
        self.conv1 = ConvBlock(3, 64, [7,7], 2, padding=3)
#         self.pool1 = ivy.max_pool2d([3,3], 2, "SAME", ceil_mode=True)

        self.conv2 = ConvBlock(64, 64, [1,1], 1, padding=0)
        self.conv3 = ConvBlock(64, 192, [3,3], 1, padding=1)
#         self.pool3 = ivy.max_pool2d([3,3], 2, "SAME", ceil_mode=True)
        
        self.inception3A = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3B = Inception(256, 128, 128, 192, 32, 96, 64)
#         self.pool4 = ivy.max_pool2d([3,3], 2, "SAME", ceil_mode=True)

        self.inception4A = Inception(480, 192, 96, 208, 16, 48, 64)

        self.aux4A = Auxiliary(512, self.num_classes)

        self.inception4B = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4C = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4D = Inception(512, 112, 144, 288, 32, 64, 64)

        self.aux4D = Auxiliary(528, self.num_classes)

        self.inception4E = Inception(528, 256, 160, 320, 32, 128, 128)
#         self.pool5 = ivy.max_pool2d([3,3], 2, "SAME", ceil_mode=True)

        self.inception5A = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5B = Inception(832, 384, 192, 384, 48, 128,128)
        self.pool6 = ivy.AdaptiveAvgPool2d([1,1])

        self.dropout = ivy.Dropout(0.4)
        self.fc = ivy.Linear(1024, self.num_classes, with_bias=False)


    def _forward(self, x):
        display(f"the input IMAGE shape should be (1, 224, 224, 3) | and it is: {ivy.shape(x)}")
        out = self.conv1(x)
        
        # maxpool2d_1
        display(f"the input shape for max_pool2d_1 should be (1, 112, 112 64) | and it is: {ivy.shape(out)}")
        out = ivy.max_pool2d(out, [3,3], 2, 0, ceil_mode=True, data_format="NCHW")
        
        display(f"the input shape for conv2 should be (1, 56, 56, 64) | and it is: {ivy.shape(x)}")
        out = self.conv2(out)
        
        display(f"the input shape for conv3 shdould be (1, 56, 56, 64) | and it is: {ivy.shape(out)}")
        out = self.conv3(out)
        
        # maxpool2d_2
        display(f"the input shape for maxpool2d_2 shdould be (1, 56, 56, 192) | and it is: {ivy.shape(out)}")
        out = ivy.max_pool2d(out, [3,3], 2, 0, ceil_mode=True, data_format="NCHW")
        
        display(f"the input shape shdould be (1, 28, 28, 192) | and it is: {ivy.shape(out)}")
        out = self.inception3A(out)
        
        display(f"the input shape shdould be (1, 28, 28, 256) | and it is: {ivy.shape(out)}")
        out = self.inception3B(out)
        
        # maxpool2d_3
        display(f"the input shape for maxpool2d_3 shdould be (1, 28, 28, 480) | and it is: {ivy.shape(out)}")
        out = ivy.max_pool2d(out, [3,3], 2, 0, ceil_mode=True, data_format="NCHW")
        
        display(f"the input shape for inception4a shdould be (1, 14, 14, 480) | and it is: {ivy.shape(out)}")
        out = self.inception4A(out)

        display(f"the input shape for aux4a shdould be (1, 14, 14, 512) | and it is: {ivy.shape(out)}")
        aux1 = self.aux4A(out)
        display(f"the output shape from aux4a shdould be (1, 1000) | and it is: {ivy.shape(out)}")
        
        
        display(f"the input shape for inception4b shdould be (1, 14, 14, 512)| and it is: {ivy.shape(out)}")
        out = self.inception4B(out)
        
        display(f"the input shape for inception4c shdould be (1, 14, 14, 512) | and it is: {ivy.shape(out)}")
        out = self.inception4C(out)
        
        display(f"the input shape for inception4 shdould be (1, 14, 14, 512) | and it is: {ivy.shape(out)}")
        out = self.inception4D(out)
        
        display(f"the input shape for aux4d shdould be (1, 14, 14, 528)  | and it is: {ivy.shape(out)}")
        aux2 = self.aux4D(out)
        display(f"the output shape from aux4a shdould be (1, 1000) | and it is: {ivy.shape(out)}")
        
        
        display(f"the input shape for inception4e should be (1, 14, 14, 528) | and it is: {ivy.shape(out)}")
        out = self.inception4E(out)
        
        # maxpool2d_4
        display(f"the input shape for maxpool2d_4 should be (1, 7, 7, 832) | and it is: {ivy.shape(out)}")
        out = ivy.max_pool2d(out, [2,2], 2, 0, ceil_mode=True, data_format="NCHW")
        
        display(f"the input shape for inception5a is (1, 7, 7, 832) | and it is: {ivy.shape(out)}")
        out = self.inception5A(out)
        
        display(f"the input shape for inception5b shdould be (1, 7, 7, 1024) | and it is: {ivy.shape(out)}")
        out = self.inception5B(out)
        
        display(f"the input shape for AdpAvgPool should be (1, 1, 1, 1024) | and it is: {ivy.shape(out)}")
#       out = ivy.reshape(out, (1, 1024, 7, 7))
        # out = ivy.permute_dims(out, (0, 3, 1, 2))
        out = self.pool6(out)
        
        display(f"the input shape for flatten layer should be (1, 1, 1, 1024) | and it is: {ivy.shape(out)}")
        out = ivy.flatten(out, start_dim=1)
        
        display(f"the input shape for dropout should be (1, 1024) | and it is: {ivy.shape(out)}")
        out = self.dropout(out)
        
        display(f"the input shape for fc layer should be (1, 1024) | and it is: {ivy.shape(out)}")
        out = self.fc(out)
        display(f"final fc output shape should be (1, 1000) | and it is: {ivy.shape(out)}")
        
        return out, aux1, aux2


In [27]:
def test_googlenet_block():
    random_test_tensor = ivy.random_normal(shape=(1, 3, 224, 224))

    display(f"random_test_tensor shape is: {random_test_tensor.shape}")
    
    auxiliary_block = Auxiliary(512, 1000)
    display(f"Auxiliary block configuration is: Auxiliary(512, 1000)")
    calc_output_shape = auxiliary_block(random_test_tensor).shape
    true_output_shape = (1, 1000)
    display(f"calc_output_shape is: {calc_output_shape} | and true_output_shape is: {true_output_shape}")
    
    assert calc_output_shape == true_output_shape;

test_auxiliary_block()

'random_test_tensor shape is: ivy.Shape(1, 512, 14, 14)'

'Auxiliary block configuration is: Auxiliary(512, 1000)'

'the input shape for adapAvgPool layer should be (1, 512, 14, 14) | and it is: ivy.Shape(1, 512, 14, 14)'

'the input shape for conv layer should be (1, 4, 4, 512) | and it is: ivy.Shape(1, 512, 4, 4)'

'the input shape for flatten layer should be (1, 4, 4, 128) | and it is: ivy.Shape(1, 128, 4, 4)'

...

'the input shape for relu layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the input shape for dropout layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the input shape for fc2 layer should be (1, 1024) | and it is: ivy.Shape(1, 1024)'

'the output shape from fc2 layer should be (1, 1000) | and it is: ivy.Shape(1, 1000)'

'calc_output_shape is: ivy.Shape(1, 1000) | and true_output_shape is: (1, 1000)'

In [26]:
def _prune_keys(raw, ref, raw_keys_to_prune=[], ref_keys_to_prune=[]):
    pruned_ref = {}
    if raw_keys_to_prune:
        raw = raw.cont_prune_keys(raw_keys_to_prune)
    if ref_keys_to_prune:
        pruned_ref = ref.cont_at_keys(ref_keys_to_prune)
        ref = ref.cont_prune_keys(ref_keys_to_prune)
    return raw, ref, pruned_ref


def _map_weights(raw, ref, custom_mapping=None):
    mapping = {}
    for old_key, new_key in zip(
        raw.cont_sort_by_key().cont_to_iterator_keys(),
        ref.cont_sort_by_key().cont_to_iterator_keys(),
    ):
        new_mapping = new_key
        if custom_mapping is not None:
            new_mapping = custom_mapping(old_key, new_key)
        mapping[old_key] = new_mapping
    return mapping


def load_torch_weights(
    url,
    ref_model,
    raw_keys_to_prune=[],
    ref_keys_to_prune=[],
    custom_mapping=None,
    map_location=torch.device("cpu"),
):
    ivy.set_backend("torch")
    weights = torch.hub.load_state_dict_from_url(url, map_location=map_location)

    weights_raw = ivy.to_numpy(ivy.Container(weights))
    weights_raw, weights_ref, pruned_ref = _prune_keys(
        weights_raw, ref_model.v, raw_keys_to_prune, ref_keys_to_prune
    )
    mapping = _map_weights(weights_raw, weights_ref, custom_mapping=custom_mapping)

    ivy.previous_backend()
    w_clean = weights_raw.cont_restructure(mapping, keep_orig=False)
    if ref_keys_to_prune:
        w_clean = ivy.Container.cont_combine(w_clean, pruned_ref)
    return ivy.asarray(w_clean)

In [ ]:
def _inceptionNet_torch_weights_mapping(old_key, new_key):
    W_KEY = ["conv/weight"]
    new_mapping = new_key
    if any([kc in old_key for kc in W_KEY]):
        new_mapping = {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
    return new_mapping


def ivy_inceptionNet_v1(pretrained=True):
    # downlaod weights => clean theme => compare the cleaned_w_structure to model_strucutre
    # so this func itslef acts like a unit test for preparing the weights for the model.
    """InceptionNet-V1 model"""
    if not pretrained:
        return GoogLeNet()

    reference_model = GoogLeNet()
#     display(reference_model.v)
    url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
    w_clean = load_torch_weights(
        url,
        reference_model,
        raw_keys_to_prune=["num_batches_tracked"],
        custom_mapping=_inceptionNet_torch_weights_mapping,
        )
    return GoogLeNet(v=w_clean)

# torch model implementation****

In [ ]:
import warnings
from collections import namedtuple
from functools import partial
from typing import Any, Callable, List, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(
        self,
        num_classes: int = 1000,
        dropout: float = 0.2,
        dropout_aux: float = 0.7,
    ) -> None:
        super().__init__()
        conv_block = BasicConv2d
        inception_block = Inception
        inception_aux_block = InceptionAux

        self.conv1 = conv_block(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
        self.conv2 = conv_block(64, 64, kernel_size=1)
        self.conv3 = conv_block(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.inception5a = inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = inception_block(832, 384, 192, 384, 48, 128, 128)

        self.aux1 = inception_aux_block(512, num_classes, dropout=dropout_aux)
        self.aux2 = inception_aux_block(528, num_classes, dropout=dropout_aux)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x: Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]:
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        return x, aux2, aux1

In [ ]:
class Inception(nn.Module):
    def __init__(
        self,
        in_channels: int,
        ch1x1: int,
        ch3x3red: int,
        ch3x3: int,
        ch5x5red: int,
        ch5x5: int,
        pool_proj: int,
        conv_block: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1), conv_block(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1),
            conv_block(ch5x5red, ch5x5, kernel_size=3, padding=1),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            conv_block(in_channels, pool_proj, kernel_size=1),
        )

    def forward(self, x: Tensor) -> List[Tensor]:
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return outputs


In [ ]:
class InceptionAux(nn.Module):
    def __init__(
        self,
        in_channels: int,
        num_classes: int,
        conv_block: Optional[Callable[..., nn.Module]] = None,
        dropout: float = 0.7,
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.conv = conv_block(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x: Tensor) -> Tensor:
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = F.adaptive_avg_pool2d(x, (4, 4))
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        # N x 1024
        x = self.dropout(x)
        # N x 1024
        x = self.fc2(x)
        # N x 1000 (num_classes)

        return x

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride=1, padding=0) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, kernel_size=kernel_size ,stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [ ]:
url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
weights = torch.hub.load_state_dict_from_url(url, map_location='cpu')

In [ ]:
torch_googlenet_re = GoogLeNet()
torch_googlenet_re.load_state_dict(weights)

# image prep

In [ ]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
    
from PIL import Image
dog_image = Image.open(filename)
display(dog_image)

In [ ]:
# Preprocess torch image
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])
torch_img = Image.open("/kaggle/input/images/cat.jpg")
torch_img = preprocess(dog_image)
torch_img = torch.unsqueeze(torch_img, 0)


In [ ]:
def load_and_preprocess_img(
    path,
    new_size,
    crop,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
    data_format="NHWC",
    to_ivy=False,
):
    img = Image.open(path)
    compose = transforms.Compose(
        [
            transforms.Resize(new_size),
            transforms.CenterCrop(crop),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ]
    )
    img = compose(img)
    img = img.unsqueeze(0)
    if data_format == "NHWC":
        img = img.permute((0, 2, 3, 1))
    return ivy.array(img.numpy()) if to_ivy else img.numpy()

In [ ]:
# Preprocess ivy image
img = load_and_preprocess_img(
        filename,
        256,
        224,
        to_ivy=True,
    )
print(img.shape)

# ivy_img = torch_img.permute((0, 2, 3, 1)).detach().numpy()

# calling models

In [ ]:
!echo -n gDNLm8VEO7QleBN5GdTcfMa7OHWBOWcyOiHWr2KI654= > .ivy/key.pem

In [ ]:
# predicting image using IVY googlenet
ivy.set_backend('torch')
ivy_googlenet = ivy_inceptionNet_v1(pretrained=True)
ivy_googlenet.compile(args=(img,))

In [ ]:
torch_googlenet_re.eval()

In [ ]:
# predicting torch_img using torch_googlenet
from torchvision.models import googlenet as torch_googlenet, GoogLeNet_Weights
# torch_googlenet = torch_googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
torch_googlenet = torch_googlenet(pretrained=True)
torch_googlenet.eval()

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# predicting

In [ ]:
output = ivy.softmax(ivy_googlenet(tuple(img))[0])  # pass the image to the model
classes = ivy.argsort(output[0], descending=True)[:3]  # get the top 3 classes
logits = ivy.gather(output[0], classes)  # get the logits

print("Indices of the top 3 classes are:", classes)
print("Logits of the top 3 classes are:", logits)
print("Categories of the top 3 classes are:", [categories[i] for i in classes.to_list()])

In [ ]:
torch_output = torch.softmax(torch_googlenet(torch_img), dim=1)
torch_classes = torch.argsort(torch_output[0], descending=True)[:3]
torch_logits = torch.take(torch_output[0], torch_classes)

print("Indices of the top 3 classes are:", torch_classes)
print("Logits of the top 3 classes are:", torch_logits)
print("Categories of the top 3 classes are:", [categories[i] for i in torch_classes])

In [ ]:
torch_googlenet_re(torch_img)

# asserting

In [ ]:
assert output.shape == tuple([1, 1000])

In [ ]:
assert np.array_equal(torch_classes, classes)

In [ ]:
assert torch.allclose(torch_logits, logits, rtol=0.5)